In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bone_Density"
cohort = "GSE56816"

# Input paths
in_trait_dir = "../../input/GEO/Bone_Density"
in_cohort_dir = "../../input/GEO/Bone_Density/GSE56816"

# Output paths
out_data_file = "../../output/preprocess/Bone_Density/GSE56816.csv"
out_gene_data_file = "../../output/preprocess/Bone_Density/gene_data/GSE56816.csv"
out_clinical_data_file = "../../output/preprocess/Bone_Density/clinical_data/GSE56816.csv"
json_path = "../../output/preprocess/Bone_Density/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression study of blood monocytes in pre- and postmenopausal females with low or high bone mineral density"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: Female'], 1: ['bone mineral density: high BMD', 'bone mineral density: low BMD'], 2: ['state: postmenopausal', 'state: premenopausal'], 3: ['cell type: monocytes']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# Check the sample characteristics to determine data availability
# 1. Gene expression data - From series title it looks like gene expression study, so it's available
is_gene_available = True

# 2. Identify rows containing trait, age, and gender data
# From sample characteristics:
# - Gender is in row 0 and all are female
# - Bone mineral density (our trait) is in row 1
# - Row 2 has menopausal state (can be used to infer age groups)
# - Age is not explicitly available

trait_row = 1  # Bone mineral density is in row 1
age_row = 2    # We can infer age from menopausal state
gender_row = None  # All subjects are female, so this is a constant

# Define conversion functions
def convert_trait(value):
    """Convert bone mineral density values to binary (0=low, 1=high)"""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'low' in value.lower():
        return 0
    elif 'high' in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Infer age category from menopausal state as a binary variable
    (0=premenopausal/younger, 1=postmenopausal/older)"""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'premenopausal' in value.lower():
        return 0
    elif 'postmenopausal' in value.lower():
        return 1
    else:
        return None

# Check if trait data is available
is_trait_available = trait_row is not None

# 3. Save metadata about the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. If trait data is available, extract clinical features
if is_trait_available:
    # The sample characteristics were given in dictionary format in the previous output
    # We need to create a DataFrame from this dictionary
    sample_characteristics = {
        0: ['gender: Female'], 
        1: ['bone mineral density: high BMD', 'bone mineral density: low BMD'], 
        2: ['state: postmenopausal', 'state: premenopausal'], 
        3: ['cell type: monocytes']
    }
    
    # Convert the dictionary to DataFrame format as expected by geo_select_clinical_features
    # Create a sample list based on the characteristics
    # We need to create all combinations of the characteristics
    samples = []
    
    # For high BMD, postmenopausal
    samples.append([sample_characteristics[0][0], sample_characteristics[1][0], sample_characteristics[2][0], sample_characteristics[3][0]])
    # For high BMD, premenopausal
    samples.append([sample_characteristics[0][0], sample_characteristics[1][0], sample_characteristics[2][1], sample_characteristics[3][0]])
    # For low BMD, postmenopausal
    samples.append([sample_characteristics[0][0], sample_characteristics[1][1], sample_characteristics[2][0], sample_characteristics[3][0]])
    # For low BMD, premenopausal
    samples.append([sample_characteristics[0][0], sample_characteristics[1][1], sample_characteristics[2][1], sample_characteristics[3][0]])
    
    # Create a DataFrame with samples as columns
    clinical_data = pd.DataFrame(samples).transpose()
    clinical_data.columns = [f'Sample_{i+1}' for i in range(len(samples))]
    
    # Make sure the directory exists before saving
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Use the function to extract and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=None
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the extracted clinical features to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'Sample_1': [1.0, 1.0], 'Sample_2': [1.0, 0.0], 'Sample_3': [0.0, 1.0], 'Sample_4': [0.0, 0.0]}
Clinical data saved to ../../output/preprocess/Bone_Density/clinical_data/GSE56816.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These look like Affymetrix probe IDs rather than standard human gene symbols
# For example, identifiers like "1007_s_at" are typical Affymetrix microarray probe IDs
# They need to be mapped to human gene symbols for better interpretability

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the gene annotation dataframe that contain probe IDs and gene symbols
# From the preview, we can see:
# - The 'ID' column in gene_annotation matches the gene identifiers in gene_data
# - The 'Gene Symbol' column contains the human gene symbols we want to map to

# 2. Extract the gene mapping dataframe with the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Examine the first few gene symbols to verify the mapping worked properly
print("After mapping from probes to genes, first 20 gene symbols:")
print(gene_data.index[:20])


After mapping from probes to genes, first 20 gene symbols:
Index(['A-', 'A-1', 'A-2', 'A-2-', 'A-3', 'A-4', 'A-52', 'A-E', 'A-I', 'A-I-',
       'A-II', 'A-IV', 'A-KAT03057', 'A-T', 'A-V', 'A0', 'A1', 'A1-', 'A1-12',
       'A1-A'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data and link with genetic data
# First inspect the clinical_df to understand its structure
clinical_df = pd.read_csv(out_clinical_data_file)
print("Clinical data columns:", clinical_df.columns)

# In Step 2, we created clinical data with the trait values in the first row
# We need to properly reshape this for linking
clinical_feat_df = pd.DataFrame()
# The first row of clinical_df contains our trait values (bone density)
clinical_feat_df[trait] = clinical_df.iloc[0].values
# The second row, if present, would contain Age values
if clinical_df.shape[0] > 1:
    clinical_feat_df['Age'] = clinical_df.iloc[1].values

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_feat_df, normalized_gene_data)

# Print data structure for debugging
print("Linked data shape:", linked_data.shape)
print("Linked data first few columns:", linked_data.columns[:10])

# 3. Handle missing values in the linked data
# Ensure the trait column exists
if trait not in linked_data.columns:
    # If trait column doesn't exist, it might be a numeric index
    # Rename first column to trait name if it exists
    if 0 in linked_data.columns:
        linked_data = linked_data.rename(columns={0: trait})
        print(f"Renamed column 0 to {trait}")

linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
note = "Dataset contains gene expression data from blood monocytes in pre- and postmenopausal females with low or high bone mineral density."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Bone_Density/gene_data/GSE56816.csv
Clinical data columns: Index(['Sample_1', 'Sample_2', 'Sample_3', 'Sample_4'], dtype='object')
Linked data shape: (75, 18295)
Linked data first few columns: Index([0, 1, 2, 3, 'A1BG', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT'], dtype='object')
Renamed column 0 to Bone_Density
Quartiles for 'Bone_Density':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Bone_Density' in this dataset is severely biased.

Abnormality detected in the cohort: GSE56816. Preprocessing failed.
Data was determined to be unusable and was not saved
